<a href="https://colab.research.google.com/github/hatemamine/Improving-Arabic-IR-and-Reranking-Performance-with-Knowledge-Distillation/blob/main/embedding_mrtydi_collection_queries_with_bi_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Improving Efficient Neural Ranking Models with Cross-Architecture Knowledge Distillation
https://arxiv.org/pdf/2010.02666
https://github.com/sebastian-hofstaetter/neural-ranking-kd
https://zenodo.org/records/4068216
https://sbert.net/docs/package_reference/sentence_transformer/losses.html#marginmseloss
https://huggingface.co/datasets/unicamp-dl/mmarco
instance Temperature Knowledge Distillation https://arxiv.org/pdf/2407.00115
https://github.com/Zhengbo-Zhang/ITKD/tree/main
ESE: Espresso Sentence Embeddings https://arxiv.org/pdf/2402.14776
Making Monolingual Sentence Embeddings Multilingual using Knowledge Distillation https://aclanthology.org/2020.emnlp-main.365.pdf
https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/multilingual/make_multilingual.py
https://www.sbert.net/examples/applications/retrieve_rerank/README.html
https://www.sbert.net/examples/training/multilingual/README.html
https://www.sbert.net/examples/training/distillation/README.html?highlight=knowledge%20distillation
https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/ms_marco/train_cross-encoder_kd.py
https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/distillation/model_distillation.py
https://www.sbert.net/docs/sentence_transformer/pretrained_models.html
https://www.sbert.net/docs/pretrained-models/msmarco-v5.html
https://huggingface.co/sentence-transformers/msmarco-bert-base-dot-v5
https://towardsdatascience.com/distillation-of-bert-like-models-the-code-73c31e8c2b0a
https://huggingface.co/cross-encoder/ms-marco-MiniLM-L-12-v2?text=I+like+you.+I+love+you
FastFormers: Highly Efficient Transformer Models for Natural Language Understanding https://arxiv.org/pdf/2010.13382
https://github.com/philschmid/huggingface-sagemaker-workshop-series/blob/9f625c2c261db564532b0f9ec0ef6581c122fec3/workshop_4_distillation_and_acceleration/scripts/knowledge_distillation.py
https://github.com/philschmid/huggingface-sagemaker-workshop-series/blob/main/workshop_4_distillation_and_acceleration/lab2_knowledge_distillation.ipynb

In [ ]:
!pip install -U datasets huggingface-hub -q
!pip install sentence_transformers -q

In [ ]:
organization_dataset_id="hatemestinbejaia/ExperimentDATA_knowledge_distillation_vs_fine_tuning"
collection_name= "mrtydi-collection-arabic-preprocess-for-araelectra"
queries_name= "mrtydi-queries-arabic-preprocess-for-araelectra"
qerl_file ="qrels.test.mrtydi.txt"
bm25_100condidatxquery ="run.mrtydi.bm25.ar.test.txt"
list_of_models = [ "hatemestinbejaia/mmarco-Arabic-AraElectra-bi-encoder-NoKD-v1",
              "hatemestinbejaia/mmarco-Arabic-AraElectra-bi-encoder-KD-v1",
              "hatemestinbejaia/mmarco-Arabic-AraDPR-bi-encoder-NoKD-v1",
              "hatemestinbejaia/mmarco-Arabic-AraDPR-bi-encoder-KD-v1",
              "hatemestinbejaia/mmarco-Arabic-mMiniLML-bi-encoder-NoKD-v1",
              "hatemestinbejaia/mmarco-Arabic-mMiniLML-bi-encoder-KD-v1",
            ]

In [ ]:
model_name= list_of_models[5]
print(model_name)
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer(model_name)
model.eval()

def embedding(batch):
    with torch.no_grad(): batch["embedding"] = model.encode(batch["text"], convert_to_tensor=True, show_progress_bar=False, batch_size=128)
    #for i in range(len(batch["text"])):batch["text"][i] = arabert_prep.preprocess(batch["text"][i])
    return batch

hatemestinbejaia/mmarco-Arabic-mMiniLML-bi-encoder-KD-v1


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/199 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/35.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
datasetC = load_dataset(organization_dataset_id, collection_name, trust_remote_code=True)
print(datasetC['train'][100])
print(datasetC)
import torch
datasetC= datasetC.map(embedding, batched=True, remove_columns=["text"])

In [ ]:
from datasets import load_dataset
datasetQ = load_dataset(organization_dataset_id, queries_name, trust_remote_code=True)
datasetQ = datasetQ["train"]
print(datasetQ)
print(datasetQ[0])
import torch
datasetQ= datasetQ.map(embedding, batched=True)
datasetQ

Dataset({
    features: ['id', 'text'],
    num_rows: 1081
})
{'id': 15513, 'text': 'كم عدد مرات فوز الأوروغواي ببطولة كاس العالم لكرو القدم ؟'}


Dataset({
    features: ['id', 'text', 'embedding'],
    num_rows: 1081
})

In [ ]:
# uncomment to push to yours huggingface repository
"""
import huggingface_hub
hf = huggingface_hub.HfFolder()
access_token = "your huggingface access token"
hf.save_token(access_token)
datasetQ.push_to_hub(organization_dataset_id, "mrTydi-queries-Arabic-embedding-"+model_name.split("/")[1])
datasetC.push_to_hub(organization_dataset_id, "mrTydi-collection-Arabic-embedding-"+model_name.split("/")[1])
"""